Этот блокнот *примерно* соответствует [главе 10.2 книги Git - Git Internals - Git Objects](https://git-scm.com/book/en/v2/Git-Internals-Git-Objects) .

# Прежде чем мы начнем, давайте настроим вашу среду

TODO: чтение из [дотфайла](https://github.com/theskumar/python-dotenv)

In [ ]:
from pprint import pprint

!sh clean.sh

%env USERNAME="<git config user.name>"
%env USEREMAIL="<git config user.email>"

# Git Internals - Git Объекты

![Сложно выглядящий объект](https://talkingwithimage.files.wordpress.com/2014/10/f5ac465df56759f1fecb01e677ceeb34.jpg)

## Git Objects

###

[Факт № 1 - Git является файловой системой с адресным содержимым](https://notebooks.azure.com/dalinwilliams/projects/git-good/html/Git%20Facts)

Это означает, что git хранит и получает данные, используя уникальный ключ.

#### Отлично ... что это значит

Давайте продолжим и создадим новый репо

In [ ]:
!git init

И используйте `git hash-object` для добавления контента в наш репозиторий

In [ ]:
!echo 'test content' | git hash-object -w --stdin

Просто так, у нас есть контент! Используя `hash-content` с флагом `-w` , мы записали `'test content'` в наш репозиторий. `--stdin` позволяет нам читать из стандартного терминала - чаще, чем из файла.

In [ ]:
!find .git/objects -type f

Значение, возвращаемое `hash-content` является ключом для извлечения данных - контрольной суммой SHA-1 содержимого + **заголовок** (вспомним **заголовок** для дальнейшего).

Мы можем получить эти данные из нашего репозитория, используя `git cat-file` и контрольную сумму.

In [ ]:
!git cat-file -p d670460b4b4aece5915caf5c68d12f560a9fe3e4

Мы даже можем показать, как git реализует контроль версий - хотя и в микроуровне

Давайте создадим файл и добавим его в наш репозиторий

In [ ]:
!echo 'version 1' > test.txt
!git hash-object -w test.txt

Давайте изменим этот файл, чтобы создать 'v2' этого файла

In [ ]:
!echo 'version 2' > test.txt
!git hash-object -w test.txt

И убедитесь, что эти новые объекты были созданы в нашей директории .git / objects.

In [ ]:
!find .git/objects -type f

Теперь мы можем удалить нашу локальную копию test.txt и использовать git для получения любой версии, которую мы хотим

In [ ]:
!git cat-file -p 83baae61804e65cc73a7201a7252750c76066a30 > test.txt
!cat test.txt

In [ ]:
!git cat-file -p 1f7a7a472abf3dd9643fd615f6da379c4acb3e3a > test.txt
!cat test.txt

Теперь мы можем получить данные, используя SHA-1 файла - ура! Однако, поскольку мы не храним файл по имени, нам нужно запомнить все SHA-1 для каждого файла и каждой версии этого файла и запомнить его имя ...

![фокус, фокус, фокус](https://www.theamericanconservative.com/wp-content/uploads/2013/09/student-studying.jpg)

Не бойся! У Git есть метод обработки этой сложности. Пока помните, что это отображение «один файл - одна версия» называется `blob` - самое низкое представление данных в git. Git может даже сказать вам, что объект, на который вы смотрите, является `blob`

In [ ]:
!git cat-file -t 1f7a7a472abf3dd9643fd615f6da379c4acb3e3a

## Объекты дерева

Древовидные объекты решают проблему запоминания пар файл-версия-SHA-1.

Объекты дерева также позволяют нам хранить группу файлов вместе.

Весь контент в репозитории git хранится в виде объектов дерева и больших двоичных объектов, причем деревья представляют записи каталогов, а большие двоичные объекты представляют содержимое файлов.

Это примерно аналогично тому, как работают системы UNIX, с деревьями, представляющими записи каталогов UNIX, и с BLOB-объектами, представляющими inode / содержимое файла.

Объект дерева может содержать одну или несколько записей, каждая из которых представляет SHA-1 большого двоичного объекта или поддерева с соответствующими им режимом, именем и типом файла.

<Опрятно - что ты имеешь в виду под деревьями?!>

![Простое представление модели данных git](https://git-scm.com/book/en/v2/images/data-model-1.png)

Хорошо, давайте создадим наше собственное дерево. Как правило, это можно сделать, взяв состояние вашего индекса или промежуточной области и записав ряд древовидных объектов, поэтому давайте разберем несколько файлов

Это может быть сделано путем создания индекса, который будет функционировать как наша промежуточная область.

We can use `git update-index` to the version of test.txt that we've already added to .git/objects. We'll need to use the `--add` flag to push the file to the staging area, and combine it with the `--cacheinfo` flag because the file we're adding is not in  our diretory.

Наконец, нам нужно указать SHA-1, имя файла и режим файла. Поскольку test.txt - это обычный файл (100755 - исполняемый файл или 120000 - символическая ссылка), мы предоставляем код 100644.

Эти режимы взяты из режимов UNIX. Вышеупомянутые три режима являются единственными допустимыми режимами для файлов, то есть больших двоичных объектов в git, хотя другие режимы доступны для подмодулей и каталогов.

In [ ]:
!git update-index --add --cacheinfo 100644 \
  83baae61804e65cc73a7201a7252750c76066a30 test.txt

We can use `git ls-files --stage` to list the files staged in .git/index

In [ ]:
!git ls-files --stage

Поскольку мы добавили файл в область подготовки, мы можем использовать `git write-tree` без флага `-w` . Это создаст объект дерева из состояния индекса - если это дерево не существует

In [ ]:
!git write-tree
!git cat-file -p d8329fc1cc938780ffdd9f94e0d364e0ea74f579

Теперь мы можем проверить, что возвращенный SHA-1 ссылается на объект git дерева.

In [ ]:
!git cat-file -t d8329fc1cc938780ffdd9f94e0d364e0ea74f579

Хорошо, давайте создадим новое дерево со второй версией test.txt + новый файл

In [ ]:
!echo 'new file' > new.txt
!git update-index --add --cacheinfo 100644 \
  1f7a7a472abf3dd9643fd615f6da379c4acb3e3a test.txt
!git update-index --add new.txt

Потрясающие! Наша область подготовки должна иметь новую версию test.txt и новый файл - new.txt.

In [ ]:
!git ls-files --stage

Давайте продолжим и напишем это дерево

In [ ]:
!git write-tree
!git cat-file -p 0155eb4229851634a0f03eb265b69f5a2d56f341

Обратите внимание: в этом дереве есть обе записи файла, а SHA-1 - это test.txt, содержащий «версию 2» ( `1f7a7a` ). Ради интереса, давайте первым деревом в качестве подкаталога в это дерево. Мы можем читать деревья в нашей области подготовки, вызывая `git read-tree`

В этом случае мы будем читать существующее дерево в нашей промежуточной области как поддерево, используя флаг `prefix`

In [ ]:
!git read-tree --prefix=bak d8329fc1cc938780ffdd9f94e0d364e0ea74f579
!git write-tree
!git cat-file -p 3c4e9cd789d88d8d89c1073707c3585e41b0e614

Итак, что только что произошло?

Если бы мы создавали новый каталог из этого нового дерева, в корневом каталоге были бы новые.txt и test.txt («версия 2»), а также папка с именем bak, которая содержала бы test.txt («версия 1»). ,

![Структура данных git после добавления папки bak](https://git-scm.com/book/en/v2/images/data-model-2.png)

## Зафиксировать объекты

Итак, у нас есть три дерева, которые представляют разные снимки нашего проекта.

Подождите - мы все еще должны помнить SHA-1 этих деревьев, чтобы вспомнить эти снимки! У нас также нет никакой информации о том, кто сохранил снимки, когда и почему они были сохранены.

Это то, что коммиты будут хранить для нас

Чтобы создать объект фиксации, нам нужно вызвать `git commit-tree` и указать единственное дерево SHA-1, и какие объекты фиксации (если таковые имеются) непосредственно его обрабатывают. Давайте начнем с первого дерева, которое мы написали

In [ ]:
!git config user.name $USERNAME
!git config --replace-all user.email $USEREMAIL
first_commit_sha_1 = !echo 'first commit' | git commit-tree d8329f 
first_commit_sha_1 = first_commit_sha_1[0]
pprint(first_commit_sha_1)

Мы сохраняем коммит SHA-1 в commit_sha_1, так как на ваш SHA-1 влияют значения конфигурации user.name и user.email.

Теперь мы можем получить новый объект коммита, используя `git cat-file`

In [ ]:
!git cat-file -p $first_commit_sha_1

Формат, который вы видите здесь, прост - коммит верхнего уровня; родитель передает (команда выше не должна иметь родителей); информация об авторе (имя, адрес электронной почты и отметка времени), пустая строка и сообщение о коммите

Теперь давайте напишем два других объекта коммита, ссылающихся на коммит, который был непосредственно перед ним.

In [ ]:
second_commit_sha_1 = !echo 'second commit' | git commit-tree 0155eb -p $first_commit_sha_1
second_commit_sha_1 = second_commit_sha_1[0]
pprint(second_commit_sha_1)
third_commit_sha_1 = !echo 'third commit' | git commit-tree 3c4e9c -p $second_commit_sha_1
third_commit_sha_1 = third_commit_sha_1[0]
pprint(third_commit_sha_1)

У нас есть три новых коммита, каждый из которых указывает на одно из трех деревьев, которые мы создали. Мы можем использовать `git log` чтобы увидеть слишком знакомую цепочку коммитов и родительских коммитов - учитывая, что мы предоставляем последний коммит SHA-1

In [ ]:
!git log --stat $third_commit_sha_1

Во-первых, побалуйте себя аплодисментами - нам удалось создать историю git без использования внешних команд.

Здесь мы делаем то, что делает git, когда запускаем `git add` и `git commit` - это мы:

1. сохраненные капли для файлов, которые изменились

1. обновить индекс (добавить файлы на сцену)

1. выписать деревья

1. выписать объекты коммитов, которые ссылаются на деревья верхнего уровня и коммиты, пришедшие непосредственно перед ними

Давайте посмотрим на все наши блестящие новые объекты Git

In [ ]:
!find .git/objects -type f

Если бы мы отобразили вышеупомянутые указатели, у нас был бы объект-граф, подобный следующему:

![Все доступные объекты в вашем каталоге git](https://git-scm.com/book/en/v2/images/data-model-3.png)

### Помните:

SHA-1 коммитов будет другим, однако SHA-1 дерева и BLOB-объектов должны быть одинаковыми

## Хранение объектов

Ранее мы упоминали, что есть заголовок, сохраняемый с каждым объектом, который мы фиксируем в вашей базе данных объектов git. Давайте подробнее рассмотрим, как вычисляется этот заголовок и как он влияет на вычисление объектов SHA-1.

Используя ruby, давайте настроим образцы данных для фиксации в нашем репо

```ruby
irb >> content = "what is up, doc?" => "what is up, doc?"
```

После получения контента, git сгенерирует заголовок. Этот заголовок будет содержать git, идентифицирующий тип объекта (в данном случае blob) и пробел, за которым следует размер в байтах содержимого, и окончательный нулевой байт.

```ruby
>> header = "blob #{content.length}\0" => "blob 16\u0000"
```

Git объединит заголовок и исходный контент. Результаты затем используются для расчета SHA-1 содержания

```ruby
>> store = header + content => "blob 16\u0000what is up, doc?" >> require 'digest/sha1' => true >> sha1 = Digest::SHA1.hexdigest(store) => "bd9dbf5aae1a3862dd1526723246b20206e5fc37"
```

Мы реализовали вышеуказанную логику в 'sha-1-example.rb'. SHA-1 будет тем же SHA-1, который будет вычислен `git hash-object` !

In [ ]:
!ruby sha-1-example.rb

In [ ]:
!echo -n "what is up, doc?" | git hash-object --stdin

Затем Git сжимает новый контент с помощью zlib.

```ruby
>> require 'zlib' => true >> zlib_content = Zlib::Deflate.deflate(store) => "x\x9CK\xCA\xC9OR04c(\xCFH,Q\xC8,V(-\xD0QH\xC9O\xB6\a\x00_\x1C\a\x9D"
```

Теперь нам нужно записать этот дефлированный контент zlib в объект на диске. Мы установим путь к подкаталогу для первых двух символов значения SHA-1, а оставшиеся 38 символов будут именами файлов в этом каталоге.

```ruby
>> path = '.git/objects/' + sha1[0,2] + '/' + sha1[2,38] => ".git/objects/bd/9dbf5aae1a3862dd1526723246b20206e5fc37" >> require 'fileutils' => true >> FileUtils.mkdir_p(File.dirname(path)) => ".git/objects/bd" >> File.open(path, 'w') { |f| f.write zlib_content } => 32
```

Мы реализовали вышеописанные шаги в blob-example.rb. Мы можем запустить скрипт ruby и проверить, не создали ли мы допустимый объект git blob

In [ ]:
!ruby blob-example.rb
!git cat-file -p bd9dbf5aae1a3862dd1526723246b20206e5fc37

Сделано и сделано 🎉

Все объекты git хранятся одинаково, только с разными типами. Заголовок будет «дерево» для объектов дерева, «коммит» для объектов коммита и т. Д.